<a href="https://colab.research.google.com/github/czapata-ande/ai-frameworks/blob/main/iris_keras_6bloques_kappa_cm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación del dataset Iris con Keras y TensorFlow
## 1. CARGA DE DATOS

In [43]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

# Cargar el dataset iris
iris = load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

## 2. PREPROCESAMIENTO

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Escalar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encoding para las clases
y_categorical = to_categorical(y)

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=1912)

## 3. CONSTRUCCIÓN DEL MODELO

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Definir la arquitectura de la red neuronal
model = Sequential([
    Dense(12, activation='relu', input_shape=(X.shape[1],)),
    Dense(6, activation='relu'),
    Dense(3, activation='softmax')  # 3 clases
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## 4. COMPILACIÓN DEL MODELO

In [46]:
# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## 5. ENTRENAMIENTO DEL MODELO

In [47]:
from tensorflow.keras.callbacks import EarlyStopping

# Callback de early stopping
early_stop = EarlyStopping(
    monitor='val_loss',      # Qué métrica observar
    patience=10,             # Número de épocas sin mejora antes de detener
    restore_best_weights=True,  # Restaurar los pesos con mejor val_loss
    verbose=1
)

# Entrenar el modelo con early stopping
history = model.fit(
    X_train, y_train,
    epochs=300,
    batch_size=8,
    validation_split=0.1,
    callbacks=[early_stop],   # Aquí se activa
    verbose=1
)

Epoch 1/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.5862 - loss: 0.9869 - val_accuracy: 0.7500 - val_loss: 0.7043
Epoch 2/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5163 - loss: 0.9043 - val_accuracy: 0.7500 - val_loss: 0.6608
Epoch 3/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6085 - loss: 0.8145 - val_accuracy: 0.7500 - val_loss: 0.6265
Epoch 4/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6737 - loss: 0.8117 - val_accuracy: 0.7500 - val_loss: 0.6055
Epoch 5/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6696 - loss: 0.8567 - val_accuracy: 0.7500 - val_loss: 0.5866
Epoch 6/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7659 - loss: 0.7497 - val_accuracy: 0.7500 - val_loss: 0.5698
Epoch 7/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8100 - loss: 0.6777 - val_accuracy: 0.7500 - val_loss: 0.5522
Epoch 8/300
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8410 - loss: 0.6653 - val_accuracy: 0.750

## 6. EVALUACIÓN DEL MODELO

In [48]:
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score
import pandas as pd

# Evaluar en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy en test: {accuracy:.4f}")

# Predicciones y métricas
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Matriz de confusión con formato tipo R
cm = confusion_matrix(y_true_classes, y_pred_classes)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
print("\nConfusion Matrix and Statistics\n")
print("            Reference")
for i, row_name in enumerate(class_names):
    row_vals = " ".join(f"{val:^10}" for val in df_cm.iloc[i])
    print(f"Prediction {row_name:<10} {row_vals}")

# Kappa
kappa = cohen_kappa_score(y_true_classes, y_pred_classes)
print(f"\nKappa: {kappa:.4f}\n")

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names))

Accuracy en test: 0.9667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step

Confusion Matrix and Statistics

            Reference
Prediction setosa         12         0          0     
Prediction versicolor     0          8          0     
Prediction virginica      0          1          9     

Kappa: 0.9495

Reporte de clasificación:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        12
  versicolor       0.89      1.00      0.94         8
   virginica       1.00      0.90      0.95        10

    accuracy                           0.97        30
   macro avg       0.96      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30

